In [1]:
import os
import gc
import time
import pickle
import Geohash as geo
import numpy as np
import pandas as pd
from collections import Counter
from math import atan , pi ,sqrt, acos
from xgboost import XGBClassifier 
import warnings
warnings.filterwarnings("ignore")

In [2]:
cache_path = '../cache/'
train_path = '../data/train.csv'
test_path = '../data/test.csv'
split_time = '2017-05-23 00:00:00'
flag = True

In [3]:
#把test分成了4份
def split_test():
    test = pd.read_csv(test_path)
    for i in range(4):
        result_path = cache_path + 'test_%d.hdf'%(i)
        if os.path.exists(result_path):
            pass
        else:
            ts = test[test.index%4==i]
            ts.to_hdf(result_path, 'w', complib='blosc', complevel=5)

In [4]:
# 获取真实标签
def get_label(data):
    result_path = cache_path + 'label.pkl'
    if os.path.exists(result_path):
        label = pickle.load(open(result_path, 'rb+'))
    else:
        train = pd.read_csv(train_path)
        val = train[(train['starttime']>= split_time)]
        label = dict(zip(val['orderid'].values, val['geohashed_end_loc']))
        pickle.dump(label, open(result_path, 'wb+'))
    data['label'] = data['orderid'].map(label)
    data['label'] = (data['label'] == data['geohashed_end_loc']).astype('int')
    return data

以下部分，为特征构造部门

In [5]:
#计算方向夹角
def get_azimuth(lon1, lat1, lon2, lat2):#lon1,测试点，#lon2基站信息
    lat1 = float(lat1);lon1 = float(lon1);lat2 = float(lat2);lon2 = float(lon2)
    dlon = lon1 - lon2
    dlat = lat1 - lat2
    hyp =sqrt(dlon**2+dlat**2)
    if hyp == 0:
        return 0
    else:
        Z= dlat/hyp
        rad =acos(Z)
        deg = 180*rad/pi
        if dlon >0:
            return deg
        elif dlon <0:
            return 360-deg
        elif dlon ==0 and dlat>0:
            return 90
        else:
            return 180

In [6]:
# 计算两点之间距离
def cal_distance(lat1,lon1,lat2,lon2):
    lat1 = float(lat1);lon1 = float(lon1);lat2 = float(lat2);lon2 = float(lon2)
    dx = np.abs(lon1 - lon2)  # 经度差
    dy = np.abs(lat1 - lat2)  # 维度差
    b = (lat1 + lat2) / 2.0
    Lx = 6371004.0 * (dx / 57.2958) * np.cos(b / 57.2958)
    Ly = 6371004.0 * (dy / 57.2958)
    L = (Lx**2 + Ly**2) ** 0.5
    return L

# 计算两点之间的欧氏距离
def get_distance(result):
    locs = list(set(result['geohashed_start_loc']) | set(result['geohashed_end_loc']))
    if np.nan in locs:
        locs.remove(np.nan)
    deloc = []
    for loc in locs:
        deloc.append((geo.decode_exactly(loc)[0],geo.decode_exactly(loc)[1]))
    loc_dict = dict(zip(locs,deloc))
    geohashed_loc = result[['geohashed_start_loc','geohashed_end_loc']].values
    
    azimuths = [];diss = [];slons = []; slats= [] ;elons = [];elats = []
    
    for i in geohashed_loc:
        lat1, lon1 = loc_dict[i[0]]
        lat2, lon2 = loc_dict[i[1]]
        
        slat,slon = geo.decode(i[0])
        elat,elon = geo.decode(i[1])
        
        slat = float(slat);slon = float(slon);
        elat = float(elat);elon = float(elon)
        
        dis = cal_distance(lat1, lon1, lat2, lon2)
        azimuth = get_azimuth(lat1,lon1,lat2,lon2)
              
        azimuths.append(azimuth);diss.append(dis);
        slons.append(slon);slats.append(slat);elons.append(elon);elats.append(elat)
        
    result['azimuth'] = azimuths;result['dis'] = diss;
    result["slats"] = slats; result["slons"] = slons;result["elons"] = elons;result["elats"] = elats
    return result

In [7]:
#根据方位角，每45度划分一个方向
def azimuth_split(result):
    cut_points = range(0,360,45)
    cut_points = cut_points + [result["azimuth"].max()]
    labels = ["0-45","45-90","90-135","135-180","180-225","225-270","270-315","315-360"]
    df = pd.cut(result["azimuth"], cut_points,labels=labels,include_lowest=True)
    df = pd.get_dummies(df,prefix="azimuth_dis")
    df = pd.DataFrame(df.values * result[["dis"]].values,columns=df.columns)
    result = pd.concat([result,df],axis=1)
    return result

In [8]:
# 获取用户历史行为次数
def get_user_count(train,result):
    user_count = train.groupby('userid',as_index=False)['geohashed_end_loc'].agg({'user_count':'count'})
    result = pd.merge(result,user_count,on=['userid'],how='left')
    return result

# 获取用户去过某个地点历史行为次数
def get_user_eloc_count(train, result):
    user_eloc_count = train.groupby(['userid','geohashed_end_loc'],as_index=False)['userid'].agg({'user_eloc_count':'count'})
    result = pd.merge(result,user_eloc_count,on=['userid','geohashed_end_loc'],how='left')
    return result

# 获取用户从某个地点出发的行为次数
def get_user_sloc_count(train,result):
    user_sloc_count = train.groupby(['userid','geohashed_start_loc'],as_index=False)['userid'].agg({'user_sloc_count':'count'})
    user_sloc_count.rename(columns={'geohashed_start_loc':'geohashed_end_loc'},inplace=True)
    result = pd.merge(result, user_sloc_count, on=['userid', 'geohashed_end_loc'], how='left')
    return result

In [9]:
# 获取用户从这个路径走过几次
def get_user_sloc_eloc_count(train,result):
    user_count = train.groupby(['userid','geohashed_start_loc','geohashed_end_loc'],as_index=False)['userid'].agg({'user_sloc_eloc_count':'count'})
    result = pd.merge(result,user_count,on=['userid','geohashed_start_loc','geohashed_end_loc'],how='left')
    return result

# 获取用户从这个路径折返过几次
def get_user_eloc_sloc_count(train,result):
    user_eloc_sloc_count = train.groupby(['userid','geohashed_start_loc','geohashed_end_loc'],as_index=False)['userid'].agg({'user_eloc_sloc_count':'count'})
    user_eloc_sloc_count.rename(columns = {'geohashed_start_loc':'geohashed_end_loc','geohashed_end_loc':'geohashed_start_loc'},inplace=True)
    result = pd.merge(result,user_eloc_sloc_count,on=['userid','geohashed_start_loc','geohashed_end_loc'],how='left')
    return result

In [10]:
# 获取目标地点的热度(目的地)
def get_eloc_count(train,result):
    eloc_count = train.groupby('geohashed_end_loc', as_index=False)['userid'].agg({'eloc_count': 'count'})
    result = pd.merge(result, eloc_count, on='geohashed_end_loc', how='left')
    
    return result

# 获取目标地点的热度(出发地地)
def get_sloc_count(train,result):
    sloc_count = train.groupby('geohashed_start_loc', as_index=False)['userid'].agg({'sloc_count': 'count'})
    result = pd.merge(result, sloc_count, on='geohashed_start_loc', how='left')
    
    return result

In [11]:
#获得某个时间段，达到某个地方，或者是从某个时间段出发的信息
def hour_loc_count(train,result):
    result['hour'] = pd.to_datetime(result['starttime']).map(lambda x: x.strftime('%H'))
    train['hour'] = pd.to_datetime(train['starttime']).map(lambda x: x.strftime('%H'))
            
    hour_sloc_count = train.groupby(["hour","geohashed_start_loc"],as_index=False)["userid"].agg({'hour_sloc_count':'count'})
    #某时，从某个地方出发的次数
    hour_eloc_count = train.groupby(["hour","geohashed_end_loc"],as_index=False)["userid"].agg({'hour_eloc_count':'count'})
    #某时，到达某地的次数
    hour_sloc_eloc_count = train.groupby(["hour","geohashed_start_loc","geohashed_end_loc"],as_index=False)["userid"].agg({'hour_sloc_eloc_count':'count'})
    #某时，某地出发，到达某地的次数
    user_hour = train.groupby(["userid","hour"],as_index=False)["userid"].agg({'hour_user_count':'count'})
    #用户从某个时间出发的次数
    user_hour_eloc = train.groupby(["userid","hour","geohashed_end_loc"],as_index=False)["userid"].agg({'user_hour_eloc':'count'})
    #用户某时到某地的次数
    user_hour_sloc = train.groupby(["userid","hour","geohashed_start_loc"],as_index=False)["userid"].agg({'user_hour_sloc':'count'})
    #用户某时从某地出发的次数
    user_hour_sloc_eloc = train.groupby(["userid","hour","geohashed_start_loc","geohashed_end_loc"],as_index=False)["userid"].agg({'user_hour_sloc_eloc':'count'})
    #用户某时从某地出发到达某地的次数
    
    result = pd.merge(result, hour_sloc_count, on=["hour","geohashed_start_loc"], how='left')
    result = pd.merge(result, hour_eloc_count, on=["hour","geohashed_end_loc"], how='left')
    result = pd.merge(result, hour_sloc_eloc_count, on=["hour","geohashed_start_loc","geohashed_end_loc"], how='left')
    result = pd.merge(result, user_hour, on=["userid","hour"], how='left')
    result = pd.merge(result, user_hour_eloc, on=["userid","hour","geohashed_end_loc"], how='left')
    result = pd.merge(result, user_hour_sloc, on=["userid","hour","geohashed_start_loc"], how='left')
    result = pd.merge(result, user_hour_sloc_eloc, on=["userid","hour","geohashed_start_loc","geohashed_end_loc"], how='left')
    
    result["hour"] = result["hour"].astype(int)
    return result  

In [12]:
#获得距离和频次的比值特征
def dis_by_count(result):
    result["dis_by_count"] = result["dis"] / (result["user_sloc_eloc_count"] + 1)**1.1
    return result    

以下部分，为样本构造部分

In [13]:
#获得训练集的basic
def get_train_basic():
    print "获得训练集样本基础"
    bs_feat = ['orderid','geohashed_end_loc']
    result_path = cache_path + 'get_train_basic_%s.hdf' %("train_bs")
    if os.path.exists(result_path) & flag:
        result = pd.read_hdf(result_path, 'w')
    else:
        train = pd.read_csv(train_path)
        val = train[(train['starttime']>= split_time)]
        #train = train[(train['starttime']< split_time)]
                
        user_end_loc = get_user_end_loc(train, val)# 根据用户历史目的地点添加样本
        user_start_loc = get_user_start_loc(train, val)# 根据用户历史起始地点添加样本
        loc_to_loc = get_loc_to_loc(train, val)# 筛选起始地点去向最多的3个地点
        bike_end_loc = get_bike_end_loc(train, val)#车ID中，多次重复去同一个地方的样本
        user_go_back = get_user_go_back(train, val)#用户存在往返的样本 
        
        result = pd.concat([user_end_loc[bs_feat],
                            user_start_loc[bs_feat],
                            loc_to_loc[bs_feat],
                            bike_end_loc[bs_feat],
                            user_go_back[bs_feat],]).drop_duplicates()
        
        val.drop(["geohashed_end_loc"],axis=1,inplace=True)
        result = pd.merge(result,val,on="orderid",how="left")
        result = result[result['geohashed_end_loc'] != result['geohashed_start_loc']]
        result = result[(~result['geohashed_end_loc'].isnull()) & (~result['geohashed_start_loc'].isnull())]
        result.to_hdf(result_path, 'w', complib='blosc', complevel=5)
    return result

##获得训练集的basic
def get_test_basic(test,i):
    print "获得测试集样本基础"
    bs_feat = ['orderid','geohashed_end_loc']
    result_path = cache_path + 'get_test_basic_%d.hdf' %(i)
    if os.path.exists(result_path) & flag:
        result = pd.read_hdf(result_path, 'w')
    else:
        train = pd.read_csv(train_path)
        #train = train[(train['starttime']< split_time)]
        
        user_end_loc = get_user_end_loc(train, test) # 根据用户历史目的地点添加样本
        user_start_loc = get_user_start_loc(train, test)# 根据用户历史起始地点添加样本 
        loc_to_loc = get_loc_to_loc(train, test)  # 筛选起始地点去向最多的3个地点
        bike_end_loc = get_bike_end_loc(train, test)#车ID中，多次重复去同一个地方的样本
        user_go_back = get_user_go_back(train, test)#用户存在往返的样本 
        # 汇总样本id
        result = pd.concat([user_end_loc[bs_feat],
                            user_start_loc[bs_feat],
                            loc_to_loc[bs_feat],
                            bike_end_loc[bs_feat],
                            user_go_back[bs_feat],])
        
        result=result.drop_duplicates()
                
        result = pd.merge(result,test,on="orderid",how="left")
        result = result[result['geohashed_end_loc'] != result['geohashed_start_loc']]
        result = result[(~result['geohashed_end_loc'].isnull()) & (~result['geohashed_start_loc'].isnull())]
        
        result.to_hdf(result_path, 'w', complib='blosc', complevel=5)
    return result

# 将用户骑行过目的的地点加入样本
def get_user_end_loc(train,sample):
    result_path = cache_path + 'user_end_loc_%d.hdf' %(train.shape[0]*sample.orderid.values[0])
    if os.path.exists(result_path) & flag:
        result = pd.read_hdf(result_path, 'w')
    else:
        user_eloc = train[['userid','geohashed_end_loc']].drop_duplicates()
        result = pd.merge(sample[['orderid','userid']],user_eloc,on='userid',how='left')
        result = result[['orderid', 'geohashed_end_loc']]
        result.to_hdf(result_path, 'w', complib='blosc', complevel=5)
    return result

# 将用户骑行过出发的地点加入样本
def get_user_start_loc(train,sample):
    result_path = cache_path + 'user_start_loc_%d.hdf' %(train.shape[0]*sample.orderid.values[0])
    if os.path.exists(result_path) & flag:
        result = pd.read_hdf(result_path, 'w')
    else:
        test = pd.read_csv(test_path)
        user_sloc = pd.concat([train[['userid', 'geohashed_start_loc']],
                               test[['userid', 'geohashed_start_loc']]],
                              axis=0).drop_duplicates()
        result = pd.merge(sample[['orderid', 'userid']], user_sloc, on='userid', how='left')
        result.rename(columns={'geohashed_start_loc':'geohashed_end_loc'},inplace=True)
        result = result[['orderid', 'geohashed_end_loc']]
        result.to_hdf(result_path, 'w', complib='blosc', complevel=5)
    return result

# 筛选起始地点去向最多的5个地点
def get_loc_to_loc(train,sample):
    result_path = cache_path + 'loc_to_loc_%d.hdf' %(train.shape[0]*sample.orderid.values[0])
    if os.path.exists(result_path) & flag:
        result = pd.read_hdf(result_path, 'w')
    else:
        sloc_eloc_count = train.groupby(['geohashed_start_loc', 'geohashed_end_loc'],as_index=False)['geohashed_end_loc'].agg({'sloc_eloc_count':'count'})
        sloc_eloc_count.sort_values('sloc_eloc_count',inplace=True)
        sloc_eloc_count = sloc_eloc_count.groupby('geohashed_start_loc').tail(5)
        
        result = pd.merge(sample[['orderid', 'geohashed_start_loc']], sloc_eloc_count, on='geohashed_start_loc', how='left')
        result = result[['orderid', 'geohashed_end_loc']]
        result.to_hdf(result_path, 'w', complib='blosc', complevel=5)
    return result

# 将车ID多次重复去一个地方的样本取出
def get_bike_end_loc(train,sample):
    result_path = cache_path + 'bike_end_loc_%d.hdf' %(train.shape[0]*sample.orderid.values[0])
    if os.path.exists(result_path) & flag:
        result = pd.read_hdf(result_path, 'w')
    else:
        bike_eloc = train.groupby(["bikeid","geohashed_end_loc"],as_index=False)["geohashed_end_loc"].agg({"bike_count":"count"})
        bike_eloc = bike_eloc[bike_eloc.bike_count>2][["bikeid","geohashed_end_loc"]]
        result = pd.merge(sample[['orderid','bikeid']],bike_eloc,on='bikeid',how='left')
        result = result[['orderid', 'geohashed_end_loc']]
        result.to_hdf(result_path, 'w', complib='blosc', complevel=5)
    return result

#将用户往返过的地方，放入样本
def get_user_go_back(train,sample):
    result_path = cache_path + 'user_go_back_%d.hdf' %(train.shape[0]*sample.orderid.values[0])
    
    if os.path.exists(result_path) & flag:
        result = pd.read_hdf(result_path, 'w')
    else:
        user_go_back = pd.merge(train[["orderid","userid","geohashed_start_loc","geohashed_end_loc"]],
                                train[["orderid","userid","geohashed_start_loc","geohashed_end_loc"]],
                                left_on=["userid","geohashed_start_loc","geohashed_end_loc"],
                                right_on=["userid","geohashed_end_loc","geohashed_start_loc"],
                                how ="inner")
        
        user_go_back = user_go_back[["userid","geohashed_start_loc_x","geohashed_end_loc_x"]].drop_duplicates()
        
        result1 = pd.merge(sample[['orderid','userid',"geohashed_start_loc"]],
                          user_go_back,
                          left_on = ["userid","geohashed_start_loc"],
                          right_on= ["userid","geohashed_start_loc_x"],
                          how="inner")
        result1.rename(columns={'geohashed_end_loc_x':'geohashed_end_loc'},inplace=True)
        
        result2 = pd.merge(sample[['orderid','userid',"geohashed_start_loc"]],
                          user_go_back,
                          left_on = ["userid","geohashed_start_loc"],
                          right_on= ["userid","geohashed_end_loc_x"],
                          how="inner")
        result2.rename(columns={'geohashed_start_loc_x':'geohashed_end_loc'},inplace=True)
        
        result = pd.concat([result1,result2],axis=0)
        
        result = result[['orderid', 'geohashed_end_loc']].drop_duplicates()
        result.to_hdf(result_path, 'w', complib='blosc', complevel=5)
    return result

In [14]:
def make_tr_data():
    print "获得训练集数据"
    result_path = cache_path + 'make_tr_data.hdf'
    if os.path.exists(result_path) & flag:
        result = pd.read_hdf(result_path, 'w')
   
    else:
        train = pd.read_csv(train_path)
        train = train[(train['starttime']< split_time)]
        
        result = get_train_basic()
        result = hour_loc_count(train,result)                                   # 处理时间相关特征
        result = get_user_count(train,result)                                   # 获取用户历史行为次数
        result = get_user_eloc_count(train, result)                             # 获取用户去过这个地点几次
        result = get_user_sloc_count(train, result)                             # 获取用户从目的地点出发过几次
        result = get_user_sloc_eloc_count(train, result)                        # 获取用户从这个路径走过几次
        result = get_user_eloc_sloc_count(train, result)                        # 获取用户从这个路径折返过几次
        result = get_eloc_count(train, result)                                  # 获取目的地点的热度(目的地)
        result = get_sloc_count(train, result)                                 
        result = get_distance(result)                                           # 获取起始点和最终地点的欧式距离                                           
        result = result.fillna(0)
        result = dis_by_count(result)
        result = azimuth_split(result)
        
        result = get_label(result)
        result.to_hdf(result_path, 'w', complib='blosc', complevel=5)
    return result

In [15]:
def make_ts_data(test,i):
    print "获得测试集数据"
    result_path = cache_path + 'make_ts_data_%d.hdf'%(i)
    if os.path.exists(result_path) & flag:
        result = pd.read_hdf(result_path, 'w')
    else:
        train = pd.read_csv(train_path)
        train = train[(train['starttime']< split_time)]
        
        result = get_test_basic(test,i)
        result = hour_loc_count(train,result)                                   # 处理时间相关特征
        result = get_user_count(train,result)                                   # 获取用户历史行为次数
        result = get_user_eloc_count(train, result)                             # 获取用户去过这个地点几次
        result = get_user_sloc_count(train, result)                             # 获取用户从目的地点出发过几次
        result = get_user_sloc_eloc_count(train, result)                        # 获取用户从这个路径走过几次
        result = get_user_eloc_sloc_count(train, result)                        # 获取用户从这个路径折返过几次
        result = get_eloc_count(train, result)                                  # 获取目的地点的热度(目的地)
        result = get_sloc_count(train, result)
        result = get_distance(result)                                           # 获取起始点和最终地点的欧式距离
        result = result.fillna(0)
        result = dis_by_count(result)
        result = azimuth_split(result)
        
        result.to_hdf(result_path, 'w', complib='blosc', complevel=5)
    return result

In [16]:
#训练样本 
def train_model():
    
    train_feat = make_tr_data()
    
    feat = ["orderid","geohashed_end_loc","geohashed_start_loc","userid","bikeid","starttime"];label = ["label"]
    predictors = [_ for _ in train_feat.columns if _ not in feat+label]
    
    print "开始训练"
    result_path = cache_path + 'clf.pkl'
    if os.path.exists(result_path):
        clf = pickle.load(open(result_path, 'rb+'))
    else:
        split_test()
        clf = XGBClassifier(objective ='binary:logistic',
                                        learning_rate = 0.1,
                                        silent=0,
                                        n_estimators= 300,
                                        max_depth = 10,
                                        min_child_weight = 2,
                                        gamma = 30,
                                        reg_alpha = 10,
                                        reg_lambda = 50,                        
                                        subsample = 0.886, 
                                        colsample_bytree = 0.886,
                                        scale_pos_weight = 10,
                                        nthread =4)

        X_train = train_feat[predictors]
        y_train = train_feat[label]

        clf.fit(X_train, y_train,
                eval_set=[(X_train,y_train)], 
                eval_metric= "auc",
                early_stopping_rounds=50)
        pickle.dump(clf, open(result_path, 'wb+'))
    return clf,predictors

In [17]:
# 分组排序
def rank(data, feat1, feat2):
    data.sort_values([feat1,feat2],inplace=True,ascending=False)
    data['rank'] = range(data.shape[0])
    min_rank = data.groupby(feat1,as_index=False)['rank'].agg({'min_rank':'min'})
    data = pd.merge(data,min_rank,on=feat1,how='left')
    data['rank'] = data['rank'] - data['min_rank']
    del data['min_rank']
    return data

In [18]:
# 对结果进行整理
def reshape(pred):
    print "组合结果"
    result = pred[["orderid","geohashed_end_loc","pred"]]
    result = rank(result,'orderid','pred')
    result = result[result['rank']<3][['orderid','geohashed_end_loc','rank']]
    result = result.set_index(['orderid','rank']).unstack()
    result.reset_index(inplace=True)
    result['orderid'] = result['orderid'].astype('int')
    result.columns = ['orderid', 0, 1, 2]
    return result

In [19]:
def get_sub():
    clf,predictors = train_model()
    results = []
    for i in range(4):
        print "进行第%d个测试集预测"%(i)
        result_path = cache_path + 'test_%d.hdf'%(i)
        test = pd.read_hdf(result_path, 'w')
        
        test_feat = make_ts_data(test,i)
                
        test_feat['pred'] = clf.predict_proba(test_feat[predictors])[:,1]
        result = reshape(test_feat)
        results.append(result)
        gc.collect()
        
    result = pd.concat(results,axis=0)
    print "测试集合并完成"
    test = pd.read_csv(test_path)[['orderid']]
    result = pd.merge(test,result,on='orderid',how='left')
    result.fillna('0',inplace=True)
    result.to_csv('../sub/result.csv',index=False,header=False)
    
    import matplotlib.pylab as plt
    plt.figure(figsize=(20,4))
    feat_imp = pd.Series(clf.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')
    plt.show()

In [20]:
if __name__ == '__main__':
    get_sub()

获得训练集数据
获得训练集样本基础
开始训练


Will train until validation_0 error hasn't decreased in 50 rounds.
[0]	validation_0-auc:0.877641
[1]	validation_0-auc:0.888368
[2]	validation_0-auc:0.895950
[3]	validation_0-auc:0.895819
[4]	validation_0-auc:0.898747
[5]	validation_0-auc:0.900748
[6]	validation_0-auc:0.901863
[7]	validation_0-auc:0.902370
[8]	validation_0-auc:0.903172
[9]	validation_0-auc:0.903491
[10]	validation_0-auc:0.903585
[11]	validation_0-auc:0.903713
[12]	validation_0-auc:0.903905
[13]	validation_0-auc:0.904008
[14]	validation_0-auc:0.904360
[15]	validation_0-auc:0.904383
[16]	validation_0-auc:0.905102
[17]	validation_0-auc:0.905231
[18]	validation_0-auc:0.905864
[19]	validation_0-auc:0.905982
[20]	validation_0-auc:0.906040
[21]	validation_0-auc:0.906151
[22]	validation_0-auc:0.906179
[23]	validation_0-auc:0.906850
[24]	validation_0-auc:0.906887
[25]	validation_0-auc:0.907100
[26]	validation_0-auc:0.907140
[27]	validation_0-auc:0.907443
[28]	validation_0-auc:0.907464
[29]	validation_0-auc:0.907500
[30]	validati

KeyboardInterrupt: 

In [ ]:
方总投诉反馈：
1、玲珑街道徐家坞208号（三幢联排），山坳内阻挡导致弱覆盖（塔下黑），4G:室内-脱网,室外-95dBm,2G:室内脱网，室外：-85dBm。
用户无安装皮，飞蜂窝解决信号问题的意向，要求村落整体覆盖。
投诉点附近约50米山顶有移动4G基站，基站是覆盖高速道路的，天馈不太适宜调整。